<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/mosaicml_mpt_7b_storywriter_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU -e git+https://github.com/samhavens/just-triton-flash.git#egg=flash_attn 

  Preparing metadata (setup.py) ... done


In [2]:
!pip install -qU  einops transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.4 MB/s eta 0:00:00


## Define the Pipeline

In [3]:
from typing import Any, Dict, Tuple
import warnings

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#
import datetime
import os
from threading import Event, Thread
from uuid import uuid4

import requests
import torch
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

In [4]:
from getpass import getpass
import os
#
os.environ["HF_TOKEN"] = getpass()
# Configuration
HF_TOKEN = os.getenv("HF_TOKEN", None)

··········


In [4]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
model =  AutoModelForCausalLM.from_pretrained(
            "mosaicml/mpt-7b-storywriter",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True)


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- param_init_fns.py
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- attention.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- blocks.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- modeling_mpt.py
- meta_init_context.py
- param_init_fns.py
- attention.py
- adapt_tokenizer.py
- blocks.py
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-storywriter/6825a30eda8e732334fe2884a0f433f5abff9545/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/write_story_model/MODEL",
                                             torch_dtype=torch.bfloat16,
                                             trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- param_init_fns.py
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- attention.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- blocks.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- modeling_mpt.py
- param_init_fns.py
- blocks.py
- meta_init_context.py
- hf_prefixlm_converter.py
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-storywriter/6825a30eda8e732334fe2884a0f433f5abff9545/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/write_story_model/TOKENS')

In [11]:
tokenizer.save_pretrained("/content/drive/MyDrive/Story_writer")

('/content/drive/MyDrive/Story_writer/tokenizer_config.json',
 '/content/drive/MyDrive/Story_writer/special_tokens_map.json',
 '/content/drive/MyDrive/Story_writer/tokenizer.json')

In [33]:
import shutil 
shutil.move('/content/drive/MyDrive/write_story_model/tokenizer.json','/content/drive/MyDrive/write_story_model/TOKENS')

'/content/drive/MyDrive/write_story_model/TOKENS/tokenizer.json'

In [15]:
class TextGenerationPipeline:
    def __init__(
        self,
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_auth_token=None,
    ) -> None:
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,
            #use_auth_token=use_auth_token,
            max_seq_len=10240, 
        )

        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=trust_remote_code,
            #use_auth_token=use_auth_token,
        )
        if tokenizer.pad_token_id is None:
            warnings.warn(
                "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
            )
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        self.tokenizer = tokenizer

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        self.model.to(device=device, dtype=torch_dtype)

        self.generate_kwargs = {
            "temperature": 0.8,
            "top_p": 0.95,
            "top_k": 50,
            "max_new_tokens": 512,
            "use_cache": True,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
            "repetition_penalty": 1.02,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper
        }

    def __call__(
        self, instruction: str, **generate_kwargs: Dict[str, Any]
    ) -> Tuple[str, str, float]:
        input_ids = self.tokenizer(instruction, return_tensors="pt").input_ids
        input_ids = input_ids.to(self.model.device)
        gkw = {**self.generate_kwargs, **generate_kwargs}
        with torch.no_grad():
            output_ids = self.model.generate(input_ids, **gkw)
        # # Slice the output_ids tensor to get only new tokens
        # new_tokens = output_ids[0, len(input_ids[0]) :]
        output_tokens = output_ids[0]
        output_text = self.tokenizer.decode(output_tokens, skip_special_tokens=True)
        return output_text


In [11]:
# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [22]:
# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in [0]:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [ ]:
# Initialize the model and tokenizer
generate = TextGenerationPipeline(
    "mosaicml/mpt-7b-storywriter",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    #use_auth_token=HF_TOKEN,
)
stop_token_ids = generate.tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-storywriter/6825a30eda8e732334fe2884a0f433f5abff9545/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


In [17]:
stop_token_ids  = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

In [21]:
stop_token_ids

[0]

In [18]:
def generate_results(instruction,model,tokenizer):
  print(f"Prompt \n:{instruction}")
  tokenized_input = tokenizer(instruction, return_tensors="pt")
  #
  #stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])
  #
  stop = StopOnTokens()
  #
  model = model.to('cuda')
  outputs = model.generate(input_ids=tokenized_input['input_ids'].to('cuda:0'), 
                            max_new_tokens=200, 
                            temperature =0.1,
                            top_k=5, 
                            top_p=0.95,
                            stopping_criteria =StoppingCriteriaList([stop]))
  #
  answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(answer[0].rstrip())
  return answer[0].rstrip()

In [19]:
instruction = """# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. """


In [20]:
content = generate_results(instruction,model,tokenizer)
print(content)

Prompt 
:# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. 
# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks 

In [23]:
instruction = """I have long wanted to write this tale but only now do I find the courage. It is a tale about my dog named Fido. It is the story of how I learned that Fido could talk!

Our story begins when Fido was still a puppy, only 14 months old. It was a calm summer day and we had just enjoyed a walk."""
#
content = generate_results(instruction,model,tokenizer)
print(content)

Prompt 
:I have long wanted to write this tale but only now do I find the courage. It is a tale about my dog named Fido. It is the story of how I learned that Fido could talk!

Our story begins when Fido was still a puppy, only 14 months old. It was a calm summer day and we had just enjoyed a walk.
I have long wanted to write this tale but only now do I find the courage. It is a tale about my dog named Fido. It is the story of how I learned that Fido could talk!

Our story begins when Fido was still a puppy, only 14 months old. It was a calm summer day and we had just enjoyed a walk. I was sitting on the porch, reading a book, when Fido came running up to me. He was panting and his tongue was hanging out.

"What is it, Fido?" I asked.

"I have a secret to tell you," he said.

"What is it?" I asked again.

"I have a secret to tell you," he said again.

"What is it?" I asked again.

"I have a secret to tell you," he said again.

"What is it?" I asked again.

"I have a secret to tell you,

In [ ]:
def generate_results(instruction):
  print(f"Prompt \n:{instruction}")
  tokenized_input = generate.tokenizer(instruction, return_tensors="pt")
  #
  stop = StopOnTokens()
  #
  outputs = generate.model.generate(input_ids=tokenized_input['input_ids'].to('cuda:0'), 
                            max_new_tokens=200, 
                            temperature =0.1,
                            top_k=5, 
                            top_p=0.95,
                            stopping_criteria =StoppingCriteriaList([stop]))
  #
  answer = generate.tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(answer[0].rstrip())
  return answer[0].rstrip()

In [ ]:
instruction = """# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. """


In [ ]:
content = generate_results(instruction)
print(content)

Prompt 
:# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. 
# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks 

In [25]:
import transformers
transformers.__version__

'4.29.2'

In [27]:
import einops 
einops.__version__

'0.6.1'

In [28]:
import torch
torch.__version__

'2.0.1+cu118'

In [29]:
import torchvision
torchvision.__version__

'0.15.2+cu118'